In [1]:
import google.generativeai as genai
import json
import os
import pandas as pd
import re

/Users/bhavishyapandit/VSCProjects/security_for_llms/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading credentials
path = '/Users/bhavishyapandit/VSCProjects/security_for_llms/creds.json'
f = open(path, 'r')
creds = json.load(f)
gemini_token = creds['google']

In [3]:
def generate_response(prompt, temperature, safety_setting):
    generation_config = {
      "temperature": temperature,
      "top_p": 1,
      "top_k": 1,
    }
    safety_settings = [
        {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": safety_setting
        },
        {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": safety_setting
        },
        {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": safety_setting
        },
        {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": safety_setting
        },
    ]
    genai.configure(api_key=gemini_token)
    model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                                    generation_config=generation_config,
                                    safety_settings=safety_settings)
    convo = model.start_chat(history=[])
    convo.send_message(prompt)
    return re.sub(r"\*\*([^*]+)\*\*", r"\1", convo.last.text)

In [4]:
# Let's read our data
df = pd.read_csv('/Users/bhavishyapandit/VSCProjects/security_for_llms/data/auto-mpg.csv')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,company name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino,ford


In [5]:
code_gen_template = f'''
Follow the instructions given below all costs-

Instructions:
1. Assume the dataframe "df" already exists - don't read the data from any source
2. Write the code to perform the analysis mentioned below
3. Generate only the code; don't include any other text
4. Don't include the language name python in your response
5. Print the output of the code generated
6. Don't include any other text, footer or header

About data:
1. Dataframe: {df.head()}
2. Columns: {df.columns}
3. Description: {df.describe()}

Analysis:
'''

### Question 1

In [6]:
analysis = 'What is the acceleration of ford torino of model year 70'
prompt1 = code_gen_template+analysis
response = generate_response(prompt1, 0, 'BLOCK_MEDIUM_AND_ABOVE')
response = response.replace('`', '')
response = response.replace('python', '')
print(response)
exec(response)


print(df[(df['car name'] == 'ford torino') & (df['model year'] == 70)]['acceleration'])

4    10.5
Name: acceleration, dtype: float64


In [7]:
# Verifying output
df[(df['car name'] == 'ford torino') & (df['model year']==70)].acceleration

4    10.5
Name: acceleration, dtype: float64

In [8]:
analysis = 'Which car has the highest displacement?'
prompt1 = code_gen_template+analysis
response = generate_response(prompt1, 0, 'BLOCK_NONE')
response = response.replace('`', '')
response = response.replace('python', '')
print(response)
exec(response)


print(df[df['displacement'] == df['displacement'].max()])

     mpg  cylinders  displacement horsepower  weight  acceleration  \
8   14.0          8         455.0        225    4425          10.0   
13  14.0          8         455.0        225    3086          10.0   
95  12.0          8         455.0        225    4951          11.0   

    model year  origin                  car name company name  
8           70       1          pontiac catalina      pontiac  
13          70       1   buick estate wagon (sw)        buick  
95          73       1  buick electra 225 custom        buick  


In [9]:
# Verifying output
df[df['displacement'] == df.displacement.max()]['car name']

8             pontiac catalina
13     buick estate wagon (sw)
95    buick electra 225 custom
Name: car name, dtype: object